In [ ]:
import openai
import tiktoken
import base64
from PIL import Image
from PyPDF2 import PdfReader
from docx import Document
import os

api_key = "YOUR API KEY HERE"
client = openai.OpenAI(api_key=api_key)
encoding = tiktoken.get_encoding("cl100k_base")
token_limit = 128000
summary_ratio = 0.5

system_message = {"role": "system", "content": "You are GPT-4 Turbo. Start with 'How may I assist you today?'"}
conversation = [system_message]

def calculate_tokens(messages):
    total_tokens = 0
    for message in messages:
        total_tokens += 4
        total_tokens += len(encoding.encode(message["content"]))
    return total_tokens + 2

def get_chatgpt_response(messages):
    try:
        response = client.chat.completions.create(
            model="gpt-4-1106-preview",
            messages=messages,
            max_tokens=4000,
            temperature=0.7
        )
        response_text = response.choices[0].message.content
        cleaned_response = response_text.replace("Understood. ", "")
        return cleaned_response
    except Exception as e:
        error_msg = f"Error: {str(e)}"
        print(error_msg)
        return error_msg

def summarize_conversation(ratio=0.5):
    current_tokens = calculate_tokens(conversation)
    target_tokens = int(current_tokens * ratio)
    summary_prompt = [
        {"role": "system", "content": f"Summarize this conversation to approximately {target_tokens} tokens while preserving key details:"},
        {"role": "user", "content": " ".join([msg['content'] for msg in conversation])}
    ]
    response = get_chatgpt_response(summary_prompt)
    return response

def process_document(file_path):
    text = read_file(file_path)
    if text.startswith("Error"): return text
    
    tokens_used = len(encoding.encode(text))
    print(f"Document tokens: {tokens_used}")
    
    max_chunk_size = 32000
    chunks = [text[i:i+max_chunk_size] for i in range(0, len(text), max_chunk_size)]
    
    full_response = ""
    for i, chunk in enumerate(chunks, 1):
        print(f"Processing chunk {i}/{len(chunks)}...")
        conversation.append({"role": "user", "content": f"Analyze this text: {chunk}"})
        response = get_chatgpt_response(conversation)
        full_response += response + "\n"
        conversation.pop()
    
    return full_response

def process_image(image_path):
    try:
        image_path = image_path.strip('"').strip("'")
        with open(image_path, "rb") as img_file:
            base64_image = base64.b64encode(img_file.read()).decode('utf-8')
        
        response = client.chat.completions.create(
            model="gpt-4-vision-preview",
            messages=[{
                "role": "user",
                "content": [
                    {"type": "text", "text": "Analyze the content of this image."},
                    {"type": "image_url", "image_url": f"data:image/jpeg;base64,{base64_image}"}
                ]
            }],
            max_tokens=4000
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error processing image: {str(e)}"

def read_file(file_path):
    try:
        file_path = file_path.strip('"').strip("'")
        
        if file_path.lower().endswith('.pdf'):
            with open(file_path, 'rb') as file:
                return "".join(page.extract_text() or "" for page in PdfReader(file).pages)
        elif file_path.lower().endswith('.docx'):
            return " ".join(paragraph.text for paragraph in Document(file_path).paragraphs)
        elif file_path.lower().endswith('.txt'):
            with open(file_path, 'r', encoding='utf-8') as file:
                return file.read()
        else:
            return f"Unsupported format: {os.path.splitext(file_path)[1]}"
    except Exception as e:
        return f"Error reading file: {str(e)}"

def start_conversation():
    print("GPT-4 Token Tracker Initialized.")
    print("Commands:")
    print("- doc <path> (Upload a document for analysis)")
    print("- image <path> (Upload an image for analysis)")
    print("- summary <ratio> (Summarize conversation, e.g., summary 0.5)")
    print("- exit/quit (Exit the session)")
    print("How may I assist you today?")  # Display initial prompt
    
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]: 
            print("Exiting conversation.")
            break
            
        print(f"You: {user_input}")  # Display user input in the terminal

        if user_input.lower().startswith("doc "):
            file_path = user_input[4:].strip()
            response = process_document(file_path)
            print(f"GPT-4: {response}")
        elif user_input.lower().startswith("image "):
            image_path = user_input[6:].strip()
            response = process_image(image_path)
            print(f"GPT-4: {response}")
        elif user_input.lower().startswith("summary "):
            try:
                ratio = float(user_input[8:].strip())
                summary = summarize_conversation(ratio)
                conversation.clear()
                conversation.append(system_message)
                conversation.append({"role": "system", "content": summary})
                print("Conversation summarized.")
            except ValueError:
                print("Invalid ratio. Use a decimal (e.g., 0.5 for 50%).")
        else:
            conversation.append({"role": "user", "content": user_input})
            current_tokens = calculate_tokens(conversation)
            print(f"Current tokens: {current_tokens}")
            
            response = get_chatgpt_response(conversation)
            conversation.append({"role": "assistant", "content": response})
            print(f"GPT-4: {response}")
            
            current_tokens = calculate_tokens(conversation)
            print(f"Updated tokens: {current_tokens}")
            
            if current_tokens > token_limit - 1000:
                print(f"Approaching limit. Summarizing to {summary_ratio*100}%.")
                summary = summarize_conversation(summary_ratio)
                conversation.clear()
                conversation.append(system_message)
                conversation.append({"role": "system", "content": summary})
                current_tokens = calculate_tokens(conversation)
                print(f"New token count after summary: {current_tokens}")

if __name__ == "__main__":
    start_conversation()
